# DISTILLED MONITORING SYSTEM

## Predictive monitoring with local caching and fallback support

### 🚀 QUICK WORKFLOW:
1. `setup()` - Initialize system and fallbacks
2. `generate_datasets()` - Generate training data (resumable)
3. `train()` - Train the distilled model
4. `test()` - Test model inference
5. `demo()` - Run monitoring demo

### 📊 MONITORING:
#### 📋 Both are pretty redundant. 
- `status()` - Check system status
- `show_progress()` - Check dataset generation progress

### 🔧 RECOVERY:
- `retry_failed()` - Retry failed generations - Legacy code. It's better to just start over. 
- `reset_progress()` - Start fresh

**Planned Data Sources:** Splunk, Jira, Confluence, IBM Spectrum Conductor, VEMKD logs from Red Hat Linux

**Datset generation Fallback Order:** Remote API → Ollama → Local Model → Static Responses

In [ ]:
# Import the system
from main_notebook import *
from config import CONFIG

CONFIG['model_name'] = "bert-base-uncased" # use local cached model instead of attempting to download. 
print("🚀 Distilled Monitoring System")
print("📊 Ready for predictive monitoring with local caching")
print(f"📁 Cache directory: {CONFIG['hf_cache_dir']}")

In [ ]:
# 1. Setup system with fallback chain
print("🚀 Setting up Distilled Monitoring System...")
print("This includes: directories, fallback systems, and progress tracking")

setup_success = setup()

if setup_success:
    print("\n✅ System setup complete!")
    print("\nNext: generate_datasets() to create training data")
else:
    print("\n❌ Setup failed. Check error messages above.")
    print("You may need to install dependencies or setup Ollama.")

In [ ]:
# 2. Check system status
status()

In [ ]:
# 3. Check dataset generation progress (if any)
print("📊 Current dataset generation progress:")
show_progress()

print("\n💡 TIPS:")
print("• First run: Will show new session")
print("• Resuming: Will show existing progress")
print("• Use reset_progress() to start fresh")
print("• Use retry_failed() to retry failed items")

In [ ]:
# use with caution.
# reset_progress()

In [ ]:
# 4. Generate the datasets. 
generate_datasets()

In [ ]:
# 5. Train the distilled model
print("🏋️ TRAINING DISTILLED MODEL")
print("="*40)
print(f"Environment: {detect_training_environment()}")
print(f"Model: {CONFIG['model_name']}")
print(f"Batch size: {CONFIG['batch_size']}")
print(f"Epochs: {CONFIG['epochs']}")
print("")
print("⚠️  Training can take way long depending on hardware")
print("")

try:
    success = train()
    if success:
        print("\n✅ Training completed!")
        print("Next: test() to test the model")
    else:
        print("\n❌ Training failed - check if datasets exist")
except Exception as e:
    print(f"\n❌ Training error: {e}")
    print("Check logs for detailed error information")

In [ ]:
# 6. Test model inference
print("🧪 TESTING MODEL INFERENCE")
print("="*40)
print("Testing with scenarios:")
print("• Normal operation")
print("• CPU spike")
print("• Memory pressure")
print("")

test_success = test()

if test_success:
    print("\n✅ Model testing successful!")
    print("Next: demo() to run monitoring demo")
else:
    print("\n❌ Testing failed - ensure model is trained")

In [ ]:
# 7. Run monitoring demo
print("🎭 MONITORING DEMO")
print("="*30)
print("Features:")
print("• Real-time metric processing")
print("• Anomaly detection")
print("• Alert generation")
print("• Recommendation engine")
print("• Dashboard display")
print("")

# Customize demo duration
DEMO_MINUTES = 3

print(f"Running {DEMO_MINUTES}-minute demo...")
print("Will inject anomalies to demonstrate detection")
print("")

try:
    demo(minutes=DEMO_MINUTES)
    print("\n✅ Demo completed!")
    print("Check exported metrics history for results")
except KeyboardInterrupt:
    print("\n⏹️  Demo stopped by user")
except Exception as e:
    print(f"\n❌ Demo error: {e}")

In [ ]:
# 8. Final system status
print("📋 FINAL SYSTEM STATUS")
print("="*40)

status()

print("\n🎉 SYSTEM COMPLETE!")
print("="*30)
print("Your distilled monitoring system includes:")
print("  ✅ Multi-model fallback system")
print("  ✅ Local caching for portability")
print("  ✅ Progress tracking and resume")
print("  ✅ Trained monitoring model")
print("  ✅ Real-time anomaly detection")
print("  ✅ Actionable recommendations")
print("")
print("🔧 NEXT STEPS:")
print("  • Integrate with your actual data sources")
print("  • Customize thresholds and alerts")
print("  • Set up continuous monitoring")
print("  • Implement feedback loops for learning")

## 🛠️ Troubleshooting & Recovery

### Common Commands:
- `status()` - Complete system status
- `show_progress()` - Dataset generation progress
- `retry_failed()` - Retry failed generation items
- `reset_progress()` - Start dataset generation fresh

### Common Issues:
- **Generation interrupted:** Just run the generation cell again
- **Failed generations:** Use `retry_failed()`
- **Want to start over:** Use `reset_progress()`
- **Memory issues:** Reduce `CONFIG['batch_size']`
- **No models available:** Check Ollama is running

### Configuration:
Modify `CONFIG` in `config.py` or use:
```python
CONFIG['language_samples'] = 2000  # Increase dataset size
CONFIG['batch_size'] = 8           # Reduce for less memory
CONFIG['epochs'] = 5               # More training epochs

In [ ]:
import os
os.environ['TRITON_DISABLE_LINE_INFO'] = '1'
os.environ['TORCH_COMPILE_DISABLE'] = '1'
from transformers import AutoTokenizer
from config import CONFIG
from distilled_model_trainer import DistilledModelTrainer

trainer = DistilledModelTrainer(CONFIG, resume_training=True)
trainer.train()  # Updates latest model or creates new if none found

In [ ]:
# Test if the moved model will work for training
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModel, AutoConfig

def test_training_compatibility():
    """Test if the moved model is ready for training."""
    model_path = "./pretrained/bert-base-uncased/"
    
    print("🧪 TESTING TRAINING COMPATIBILITY")
    print("=" * 40)
    
    try:
        # Load exactly as the training code will
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            local_files_only=True
        )
        
        print("Loading config...")
        config = AutoConfig.from_pretrained(
            model_path,
            local_files_only=True
        )
        
        print("Loading model...")
        model = AutoModel.from_pretrained(
            model_path,
            config=config,
            local_files_only=True,
            torch_dtype=torch.float32
        )
        
        print("✅ All components loaded successfully!")
        
        # Test training-specific functionality
        print("\nTesting training features...")
        
        # Check if model can be put in training mode
        model.train()
        print("✅ Model can enter training mode")
        
        # Test gradient computation
        model.eval()
        test_input = tokenizer("test", return_tensors="pt", max_length=128, truncation=True)
        
        # Enable gradients
        for param in model.parameters():
            param.requires_grad = True
        
        output = model(**test_input)
        loss = output.last_hidden_state.mean()  # Dummy loss
        loss.backward()
        
        print("✅ Gradient computation works")
        
        # Check model size
        param_count = sum(p.numel() for p in model.parameters())
        print(f"✅ Model parameters: {param_count:,} ({param_count/1e6:.1f}M)")
        
        # Check config details
        print(f"✅ Hidden size: {config.hidden_size}")
        print(f"✅ Vocab size: {config.vocab_size}")
        
        print(f"\n🎉 MODEL IS READY FOR TRAINING!")
        return True
        
    except Exception as e:
        print(f"❌ Training compatibility test failed: {e}")
        return False

# Run the test
success = test_training_compatibility()

if success:
    print(f"\n✅ Your moved model in ./pretrained/bert-base-uncased/ is ready!")
    print(f"The distilled_model_trainer.py should now work without internet.")
else:
    print(f"\n❌ There may still be issues with the moved model.")

In [ ]:
# Recovery and troubleshooting commands
print("🔧 RECOVERY COMMANDS")
print("="*30)

print("\n📊 Progress Management:")
print("show_progress()    # Check current progress")
print("retry_failed()     # Retry failed items")
print("reset_progress()   # Start completely fresh")

print("\n🔍 Diagnostics:")
print("status()           # Complete system status")

print("\n⚙️  Current Configuration:")
print(f"Language samples: {CONFIG['language_samples']}")
print(f"Metrics samples: {CONFIG['metrics_samples']}")
print(f"Batch size: {CONFIG['batch_size']}")
print(f"Model: {CONFIG['model_name']}")
print(f"Cache dir: {CONFIG['hf_cache_dir']}")

print("\n💡 To modify configuration:")
print("CONFIG['language_samples'] = 2000")
print("CONFIG['batch_size'] = 8")

In [ ]:
# Optional: Run individual recovery commands
# Uncomment as needed:

# show_progress()      # Check progress
# retry_failed()       # Retry failed items
# reset_progress()     # Start fresh (WARNING: deletes progress)

print("Uncomment commands above as needed for recovery")